In [1]:
import warnings

from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
%%capture

from ding0.core import NetworkDing0
from ding0.tools.logger import setup_logger
from ding0.tools.results import save_nd_to_pickle
from ding0.tools.plots import plot_mv_topology
from egoio.tools import db
from sqlalchemy.orm import sessionmaker
import oedialect

from ding0.tools import results # to load pickl file


# create new network
nd = NetworkDing0(name='network')


# set ID of MV grid district
mv_grid_districts = [40] # fn 40, 912

#### SET DING0 SETTINGS

In [3]:
# TODO set in config file
ding0_default=False

In [4]:
engine = db.connection(section='oedb_dialect', readonly=True)
session = sessionmaker(bind=engine)()

In [5]:
create_lvgd_geo_method='convex_hull' # off, convex_hull

In [6]:
%%time

# STEP 1
nd.import_mv_grid_districts(session, mv_grid_districts_no=mv_grid_districts,
                            ding0_default=ding0_default, need_parameterization=True,
                            create_lvgd_geo_method=create_lvgd_geo_method)

Finding connected graph, iteration 1 of max. 4.
Finding connected graph, number of unconnected nodes is 3.
Finding connected graph, iteration 2 of max. 4.
Finding connected graph, number of unconnected nodes is 3.
Finding connected graph, iteration 3 of max. 4.
Finding connected graph, number of unconnected nodes is 3.
Finding connected graph, iteration 4 of max. 4.
Finding connected graph, number of unconnected nodes is 3.
Finding connected graph, max. iterations 4 trespassed. Break.
Simplified graph: 2 to 2 nodes, 2 to 2 edges
Simplified graph: 3 to 2 nodes, 4 to 2 edges
all clusters are in range
Graph already fully connected.
Simplified graph: 63 to 8 nodes, 120 to 10 edges
Simplified graph: 6 to 4 nodes, 10 to 6 edges
all clusters are in range
Graph already fully connected.
Simplified graph: 35 to 4 nodes, 66 to 4 edges
Simplified graph: 6 to 2 nodes, 10 to 2 edges
all clusters are in range
Finding connected graph, iteration 1 of max. 4.
Finding connected graph, number of unconnect

bicycle_parking is not matched in config and will be categorized as leftover
bicycle_parking is not matched in config and will be categorized as leftover


Simplified graph: 716 to 479 nodes, 1484 to 1010 edges
at least one node trespasses dist to substation.                                        cluster again with n_clusters+=1
after increment; n_cluster 5
all clusters are in range
Finding connected graph, iteration 1 of max. 4.
Finding connected graph, number of unconnected nodes is 0.
Simplified graph: 27 to 16 nodes, 48 to 26 edges
Simplified graph: 64 to 35 nodes, 142 to 84 edges
all clusters are in range
Finding connected graph, iteration 1 of max. 4.
Finding connected graph, number of unconnected nodes is 7.
Finding connected graph, iteration 2 of max. 4.
Finding connected graph, number of unconnected nodes is 5.
Finding connected graph, iteration 3 of max. 4.
Finding connected graph, number of unconnected nodes is 5.
Finding connected graph, iteration 4 of max. 4.
Finding connected graph, number of unconnected nodes is 5.
Finding connected graph, max. iterations 4 trespassed. Break.
Simplified graph: 80 to 37 nodes, 166 to 80 edg

shelter is not matched in config and will be categorized as leftover


Simplified graph: 366 to 268 nodes, 770 to 574 edges
all clusters are in range
Graph already fully connected.
Simplified graph: 22 to 8 nodes, 36 to 8 edges
Simplified graph: 11 to 9 nodes, 20 to 16 edges
all clusters are in range
ways_sql_df.empty. No ways found in MV mv_grid_district_40, LA 4670
Graph already fully connected.
Simplified graph: 21 to 4 nodes, 38 to 4 edges
Simplified graph: 41 to 23 nodes, 88 to 52 edges
all clusters are in range
Graph already fully connected.
Simplified graph: 56 to 6 nodes, 108 to 8 edges
buildings_w_loads_df.empty. No buildings found in MV mv_grid_district_40, LA 4735
Finding connected graph, iteration 1 of max. 4.
Finding connected graph, number of unconnected nodes is 0.
Simplified graph: 29 to 15 nodes, 56 to 28 edges
Simplified graph: 5 to 4 nodes, 8 to 6 edges
all clusters are in range
Graph already fully connected.
Simplified graph: 18 to 4 nodes, 32 to 4 edges
Simplified graph: 15 to 9 nodes, 28 to 16 edges
all clusters are in range
Graph al

Wall time: 2min 12s


In [7]:
# STEP 2
nd.import_generators(session, debug=False)

Generator Generator_mvgd_40_lvgd_4347000015_solar_ground_mounted_232522 cannot be assigned to non-existent LV Grid District and was allocated to a random LV Grid District (lv_grid_district_4347000015).
Generator Generator_mvgd_40_lvgd_4347000142_gas_232523 cannot be assigned to non-existent LV Grid District and was allocated to a random LV Grid District (lv_grid_district_4347000142).
Generator Generator_mvgd_40_lvgd_4347000122_hydro_232524 has no la_id and was assigned to a random LV Grid District (lv_grid_district_4347000122).
Generator Generator_mvgd_40_lvgd_4347000247_hydro_232525 has no la_id and was assigned to a random LV Grid District (lv_grid_district_4347000247).
Generator Generator_mvgd_40_lvgd_5511000000_solar_ground_mounted_232526 cannot be assigned to non-existent LV Grid District and was allocated to a random LV Grid District (lv_grid_district_5511000000).
Generator Generator_mvgd_40_lvgd_4347000149_solar_ground_mounted_232527 cannot be assigned to non-existent LV Grid Di

In [9]:
%%time

# STEP 3
nd.mv_parametrize_grid(debug=False)

Wall time: 19.7 s


In [10]:
# STEP 4: Validate MV Grid Districts
msg = nd.validate_grid_districts()

In [11]:
# STEP 5: Build LV grids
la = nd.build_lv_grids()

id_db 4488
id_db 4488000000
id_db 4488000001
id_db 4488000002
id_db 4488000003
id_db 4488000004
id_db 4488000005
id_db 4488000006
id_db 4488000007
id_db 4488000008
id_db 4488000009


In [12]:
station_point = nd._mv_grid_districts[0].mv_grid.station().geo_data
station_point.wkt

'POINT (9.466763093312689 47.6747788924774)'

In [13]:
from shapely.ops import transform
from pyproj import Transformer


In [14]:
# WGS84 (conformal) to ETRS (equidistant) projection
proj_source = Transformer.from_crs("epsg:4326", "epsg:3035", always_xy=True).transform

station_point_3035 = transform(proj_source, station_point)
station_point_3035.wkt

'POINT (4280932.702345714 2729162.472624991)'

In [15]:
la.geo_centre.wkt

'POINT (4276487.73859479 2730828.51596992)'

In [19]:
from shapely.geometry import LineString

ls = LineString([station_point_3035, la.geo_centre])
ls.length

4746.936188133519

In [18]:
g = []
for lvgd in la._lv_grid_districts:
    g.append(lvgd.graph_district)

In [19]:
len(la._lv_grid_districts), len(g)

(10, 10)

In [27]:
if la._lv_grid_districts[4].graph_district:
    print('has a g')

has a g


In [13]:
lvgd.buildings

,category,area,geometry,raccordement_building,name,number_households,n_amenities_inside,x,y,capacity,nn,nn_dist,nn_coords,cluster
osm_id,,,,,,,,,,,,,,
121254826,residential,1063.060,"POLYGON ((4284384.933772514 2725229.425153703,...",POINT (4284397.64322427 2725231.60730715),Don-Bosco-Haus (Tagungshaus),1.0,1,4.284398e+06,2.725232e+06,1.708470,29585231,46.640970,POINT (4284444.248370417 2725229.779617371),0.0
285821993,residential,650.438,"POLYGON ((4284435.943428256 2725276.335724954,...",POINT (4284448.99985888 2725267.609138271),None,1.0,1,4.284449e+06,2.725268e+06,1.708470,2923114678,26.262987,POINT (4284472.956745268 2725256.847536233),0.0
971421893,chapel,236.021,"POLYGON ((4284406.968089831 2725240.447235622,...",POINT (4284414.241857724 2725234.829397242),None,1.0,1,4.284414e+06,2.725235e+06,0.023602,29585231,30.428458,POINT (4284444.248370417 2725229.779617371),0.0


In [19]:
lvgd.buildings.loc[(lvgd.buildings.capacity < 100)]

,category,area,geometry,raccordement_building,name,number_households,n_amenities_inside,x,y,capacity,nn,nn_dist,nn_coords,cluster
osm_id,,,,,,,,,,,,,,
121254826,residential,1063.060,"POLYGON ((4284384.933772514 2725229.425153703,...",POINT (4284397.64322427 2725231.60730715),Don-Bosco-Haus (Tagungshaus),1.0,1,4.284398e+06,2.725232e+06,1.708470,29585231,46.640970,POINT (4284444.248370417 2725229.779617371),0.0
285821993,residential,650.438,"POLYGON ((4284435.943428256 2725276.335724954,...",POINT (4284448.99985888 2725267.609138271),None,1.0,1,4.284449e+06,2.725268e+06,1.708470,2923114678,26.262987,POINT (4284472.956745268 2725256.847536233),0.0
971421893,chapel,236.021,"POLYGON ((4284406.968089831 2725240.447235622,...",POINT (4284414.241857724 2725234.829397242),None,1.0,1,4.284414e+06,2.725235e+06,0.023602,29585231,30.428458,POINT (4284444.248370417 2725229.779617371),0.0


In [12]:
lvgd.lv_grid.__dict__

{'network': network,
 'id_db': 2128000000,
 'grid_district': lv_grid_district_2128000000,
 '_cable_distributors': [],
 '_loads': [],
 '_generators': [],
 'v_level': 400.0,
 '_graph': <networkx.classes.graph.Graph at 0x1fb3e30d160>,
 'default_branch_kind': 'cable',
 '_station': LVStation_mvgd_40_lvgd_2128000000,
 'population': None}

In [17]:
import osmnx as ox
import networkx as nx
edges = ox.graph_to_gdfs(lvgd.graph_district, nodes=False, edges=True)
edges

,,,length,highway,osmid,geometry
u,v,key,,,,
29585231,2923114678,0,39.462243,residential,37528126,"LINESTRING (4284444.248 2725229.780, 4284454.9..."
2923114678,29585231,0,39.462243,residential,37528126,"LINESTRING (4284444.248 2725229.780, 4284454.9..."


In [19]:
%%time

edges = ox.graph_to_gdfs(nx.MultiGraph(lvgd.graph_district), nodes=False, edges=True)
edges

Wall time: 43 ms


,,,length,highway,osmid,geometry
u,v,key,,,,
29585231,2923114678,0,39.462243,residential,37528126,"LINESTRING (4284444.248 2725229.780, 4284454.9..."


In [21]:
%%time

gg = ox.utils_graph.get_undirected(lvgd.graph_district)
edges = ox.graph_to_gdfs(gg, nodes=False, edges=True)
edges

Wall time: 164 ms


,,,length,highway,osmid,geometry,from,to
u,v,key,,,,,,
29585231,2923114678,0,39.462243,residential,37528126,"LINESTRING (4284444.248 2725229.780, 4284454.9...",2923114678,29585231
